In [1]:
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline
import processFile
import math
import pandas as pd

In [2]:
# enter sensor type (pressure, accelerometer, humidity, magnetometer, temperature)
value_type = "accelerometer"

# enter file name
file = 'C:\Users\Ludovic\Documents\Capstone\mesures\Bedroom\\mattress_test.csv'

# indicate if file type is new (with sensor id) or old (first version of the csv files): True or False
new = True

In [3]:
unit = {}
unit['temperature'] = 'C'
unit['accelerometer'] = 'G'
unit['humidity'] = 'rH'
unit['magnetometer'] = 'uT'
unit['pressure'] = 'mbar'

time, data = processFile.openFile(file, value_type, new, False)

Opening file...


In [36]:
fig, ax = plt.subplots(1)

if value_type in {"magnetometer"}:
    dataX = data[:,0]
    dataY = data[:,1]
    dataZ = data[:,2]
    dataN = np.sqrt(dataX*dataX+dataY*dataY+dataZ*dataZ)
    
    plt.plot(time, dataN, color='r', label = 'Norm of magnetic field')  #absolute value of the Y axis of magnetometer
    
elif value_type == "accelerometer":
    accelerometerX = data[:,0]
    accelerometerY = data[:,1]
    accelerometerZ = data[:,2]
    # plt.plot(time, accelerometerX, color='r', label = value_type+'X')
    # plt.plot(time, accelerometerY, color='b', label = value_type+'Y')
    # plt.plot(time, accelerometerZ, color='g', label = value_type+'Z')
    dataN = np.sqrt(accelerometerX*accelerometerX+accelerometerY*accelerometerY+accelerometerZ*accelerometerZ)
    plt.plot(time, dataN , color='r', label = value_type+'Norm')

elif value_type == "gyroscope":
    gX = data[:,0]
    gY = data[:,1]
    gZ = data[:,2]
    # plt.plot(time, accelerometerX, color='r', label = value_type+'X')
    # plt.plot(time, accelerometerY, color='b', label = value_type+'Y')
    # plt.plot(time, accelerometerZ, color='g', label = value_type+'Z')

    dataN = np.sqrt(gX*gX+gY*gY+gZ*gZ) 
    plt.plot(time, dataN, color='r', label = value_type+'Norm')

elif value_type == "temperature":
    ambient_temp = data[:,0]
    object_temp = data[:,1]

    plt.plot(time, ambient_temp, color='r', label = 'ambient_temp')
    plt.plot(time, object_temp, color='b', label = 'object_temp')

else:
    plt.plot(time, data, color='r', label = value_type)


plt.xlabel('Time (minutes)')
plt.ylabel(value_type.capitalize() + ' (' + unit[value_type] + ')')
plt.legend(loc='best')
plt.title('Simulations of pillbox usage')
plt.show()

In [5]:
x = time
y = dataN

In [6]:
# define the short and long time windows 
window_short = 10
window_long_factor = 5 # how many times longer is going to be the long window

In [7]:
# use STA/LTA Triggering
# The STA/LTA ratio method compares the average energy in a short term average (STA)
# leading window to that in a long term average (LTA) trailing window.

In [8]:
temp = list(zip(x,y))  #temporary list to prepare for the data frame (using pandas)
df = pd.DataFrame(data = temp, columns = ['Time', 'Data']) #data frame (using pandas)

In [9]:
temp = [{'time': 0, 'value': 0, 'event': 0, 'STA': 0, 'LTA': 0,'STA_LTA': 0}]

# will contain 3 columns; time = time of the time window, 
# value = mean value in the time window, event = dummy variable (1 if event, 0 otherwise)
results = pd.DataFrame(temp)  

In [10]:
foo = df.copy()
print foo.shape

(800584, 2)


In [11]:
print foo.shape
df = foo[foo['Data'] != 0]
print df.shape
print df.head(2)

(800584, 2)
(661499, 2)
       Time      Data
0  0.000000  1.131689
1  0.017083  1.131417


In [12]:
plt.plot(df.Time, df.Data , color='r', label = value_type+'Norm')
plt.xlabel('Time (minutes)')
plt.ylabel(value_type.capitalize() + ' (' + unit[value_type] + ')')
plt.legend(loc='best')
plt.title('Simulations of pillbox usage')
plt.show()

In [13]:
results

,LTA,STA,STA_LTA,event,time,value
0,0,0,0,0,0,0


In [14]:
# substract mean value of signal
mu = np.mean(df.Data)
df.Data = df.Data - [mu]*len(df.Data)
df.Data = np.abs(df.Data)

C:\Anaconda2\lib\site-packages\pandas\core\generic.py:2387: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [15]:
plt.plot(df.Time, df.Data , color='r', label = value_type+'Norm')
plt.xlabel('Time (minutes)')
plt.ylabel(value_type.capitalize() + ' (' + unit[value_type] + ')')
plt.legend(loc='best')
plt.title('Abs value of deviation')
plt.show()

In [16]:
timer = 0 # used in the for loop
for i in range(1,int(math.floor(max(time))/window_short)):
    # the loop goes through the entire file
    # temp contains the data for only one time window
    temp = df[(df['Time'] > timer) & (df['Time'] < timer + window_short)]
    if len(temp) >0:
        results.loc[i] = [0,0,0,0,np.mean(temp.Time), np.max(temp.Data)]
    timer = timer + window_short

In [17]:
results.head(10)

,LTA,STA,STA_LTA,event,time,value
0,0,0,0,0,0.000000,0.000000
1,0,0,0,0,5.008122,0.058635
2,0,0,0,0,15.007440,0.071102
3,0,0,0,0,25.006716,0.501803
4,0,0,0,0,35.006002,0.060979
5,0,0,0,0,45.005297,0.060530
6,0,0,0,0,55.004607,0.058241
7,0,0,0,0,65.003902,0.058201
8,0,0,0,0,75.003200,0.060994
9,0,0,0,0,85.002507,0.176761


In [18]:
percent = 1.2

In [19]:
results.iloc[2,:][1]

0.0

In [20]:
sub = results[0:100]
sub.head(10)

,LTA,STA,STA_LTA,event,time,value
0,0,0,0,0,0.000000,0.000000
1,0,0,0,0,5.008122,0.058635
2,0,0,0,0,15.007440,0.071102
3,0,0,0,0,25.006716,0.501803
4,0,0,0,0,35.006002,0.060979
5,0,0,0,0,45.005297,0.060530
6,0,0,0,0,55.004607,0.058241
7,0,0,0,0,65.003902,0.058201
8,0,0,0,0,75.003200,0.060994
9,0,0,0,0,85.002507,0.176761


In [21]:
sub = results
count = 0
temp =[]
for index,row in sub.iterrows():
    if count == 0:
        for i in range(0,window_long_factor):
            #temporary value, stores the value of the 10-ith mean value before the current time
            temp.append(sub.iloc[i,:][5])
    if count > window_long_factor-1:
        STA = row[5] # STA is not included in the temp list
        LTA = np.mean(temp)
        sub.STA[index] = STA
        sub.LTA[index] = LTA
        sub.STA_LTA[index] = STA/float(LTA)
        #print count, STA/float(LTA)
        if STA/float(LTA) > percent:
            #print count
            sub.event[index] = 1
        for i in range(0,window_long_factor-1):
            temp[i] = temp[i+1]
        temp[window_long_factor-1] = STA
    count = count + 1


In [22]:
print sub[sub['event'] == 1]

           LTA       STA    STA_LTA  event          time     value
9     0.059789  0.176761   2.956404      1     85.002507  0.176761
19    0.062327  0.080024   1.283931      1    184.995539  0.080024
22    0.064291  0.100595   1.564679      1    214.993442  0.100595
23    0.072575  0.186582   2.570881      1    224.992754  0.186582
29    0.067841  0.082291   1.212994      1    285.005280  0.082291
45    0.062841  0.076769   1.221644      1    444.994233  0.076769
46    0.064516  0.153416   2.377956      1    454.993565  0.153416
63    0.060935  0.079488   1.304474      1    624.998611  0.079488
140   0.060774  0.345435   5.683899      1   1394.994891  0.345435
142   0.120121  0.324355   2.700228      1   1414.993496  0.324355
161   0.060825  0.078772   1.295067      1   1604.996902  0.078772
172   0.063555  0.116456   1.832362      1   1715.005898  0.116456
181   0.062094  0.102507   1.650832      1   1804.999685  0.102507
188   0.066209  0.118681   1.792524      1   1874.994946  0.11

In [23]:
plt.plot(df.Time, df.Data , color='r', label = value_type+'Norm')
plt.xlabel('Time (minutes)')
plt.ylabel(value_type.capitalize() + ' (' + unit[value_type] + ')')
plt.legend(loc='best')
plt.show()

In [24]:
plt.plot(results.time, results.event , color='r', label = "events")
plt.xlabel('Time (minutes)')
plt.ylabel(value_type.capitalize() + ' (' + unit[value_type] + ')')
plt.legend(loc='best')
plt.show()

In [25]:
count = 0
foo =[]
for i in range(0,len(results.event)):
    if results.event.iloc[i]>0:
        count = 0
    else:
        count = count +1
    foo.append(count)

In [26]:
plt.plot(foo)

In [27]:
results_copy = results.copy()

In [28]:
# 2 possibilities: detect "ïn bed" or detect "not in bed"
# blending algorithm

In [29]:
results_copy.head(30)

,LTA,STA,STA_LTA,event,time,value
0,0.000000,0.000000,0.000000,0,0.000000,0.000000
1,0.000000,0.000000,0.000000,0,5.008122,0.058635
2,0.000000,0.000000,0.000000,0,15.007440,0.071102
3,0.000000,0.000000,0.000000,0,25.006716,0.501803
4,0.000000,0.000000,0.000000,0,35.006002,0.060979
5,0.138504,0.060530,0.437028,0,45.005297,0.060530
6,0.150610,0.058241,0.386702,0,55.004607,0.058241
7,0.150531,0.058201,0.386641,0,65.003902,0.058201
8,0.147951,0.060994,0.412260,0,75.003200,0.060994
9,0.059789,0.176761,2.956404,1,85.002507,0.176761


In [30]:
range_in_future = 10

In [31]:
results_copy.event.iloc[0:(1+range_in_future)]

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     1
10    0
Name: event, dtype: float64

In [32]:
range_in_future = 20
for i in range(1,len(results_copy)):
    if results_copy.event.iloc[i] == 0:
        if (np.sum(results_copy.event.iloc[i:(i+range_in_future)]) >0) and (np.sum(results_copy.event.iloc[i-1]) >0):
            results_copy.event.iloc[i] = .5

In [33]:
np.sum(results_copy.event.iloc[i:(i+range_in_future)])

0.0

In [35]:
plt.plot(results_copy.time, results_copy.event , color='r', label = "events")
plt.xlabel('Time (minutes)')
plt.ylabel(value_type.capitalize() + ' (' + unit[value_type] + ')')
plt.legend(loc='best')
plt.show()